## Text Classification

###  Creation of X and Y data set

In [5]:
import numpy as np
from scipy.sparse import *
from twitter_processing import *
from coocme import *
from glove_emb import *
from dataXY import *
import pickle
import nltk
import pandas as pd

In [2]:
## ---- if total file not created (for full):
filenames = ['twitter-datasets/train_neg_full.txt', 'twitter-datasets/train_pos_full.txt']
with open('twitter-datasets/train_total_full.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)
## ----- for small                
filenames = ['twitter-datasets/train_neg.txt', 'twitter-datasets/train_pos.txt']
with open('twitter-datasets/train_total.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

### Total tweets (neg, pos)  (for  small file)

In [6]:
### each line : sperate word of the tweet
###count_test gives: all word with all occurences

###--- TRAIN DATA (must be same length neg and pos)
tweets_data_tot,count_test_tot=preprocess_nltk("twitter-datasets/train_total.txt")
## for full:
#tweets_data_tot,count_test_tot=preprocess_nltk("twitter-datasets/train_total_full.txt")

###---TEST DATA
tweets_data_test,count_test_tot=preprocess_nltk("twitter-datasets/test_data.txt")


##occurences gives all words and the occurences 
## but only that occure more than 5 time
## weighted gives: the weight of each word in total word
tot_occurences, tot_weighted=occurence(count_test_tot,5)


#### Create Vocabulary

In [ ]:
## vocab gives a list of all the word
### tot_vocab is  a dic with word and value associated
tot_vocab=create_vocab(tot_occurences,len(tot_occurences))

## to get only the words: tot_vocab.keys()

# Save a dictionary into a pickle file.
with open('vocab_nltk.pkl', 'wb') as f:
        pickle.dump(tot_vocab, f, pickle.HIGHEST_PROTOCOL)   

#### Co-occurence Matrix

In [ ]:
## create co-occurence matrix (with number of same word in tweet-1 in diago)
cooc,cooc_m=create_cooc(tweets_data_tot,tot_vocab)
## save as pickle file
with open('cooc_nltk.pkl', 'wb') as f:
    pickle.dump(cooc, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
## to see data fram of cooc
voc_nice=[tot_occurences[i][0] for i in range(len(tot_occurences)) ]
cooc_m=cooc.toarray()
## print a data frame
cooc_df = pd.DataFrame(cooc_m, index = voc_nice)
cooc_df.columns = voc_nice
cooc_df

#### GloVe

In [ ]:
#create the word embeddings: [N,D]
# D is the embedding dim : default 20
w_emb=glove_emb(cooc,15,20)
np.save('embeddings_nltk.npy',w_emb)

##### Data Creation

In [3]:
##--- From here COOC and vocabulary must be created
##Create the train data  X & Y for DNN & CNN  in "tweets_nltk_dnn / cnn" & "tweets_nltk_tot_sol"
so
create_alloutput(tweets_data_tot,'tweets_nltk_tot')
##Create the test data X_test for DNN & CNN in "tweets_nltk_test_dnn / cnn"
create_test(tweets_data_test,'tweets_nltk')

# WORD2VEC REPRESENTATION

In [1]:
from sklearn.cross_validation import train_test_split
from gensim.models.word2vec import Word2Vec


# load our tweets and labels (-1 for neg and +1 for pos) associated 
X = tweets_data_tot
y = load('tweets_sol_nltk.npy')


# number of features of the vector of our word representation
dim = 300


# Initialize model
word2vec = Word2Vec(size = dim, min_count = 10)

# Build vocab
word2vec.build_vocab(X)


# Train the model
word2vec.train(X)

/Users/TomTom/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/TomTom/anaconda/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


NameError: name 'tweets_data_tot' is not defined

In [ ]:
# create vector representation of each tweet by 
def TweetVector(text, size):
    
    vec = np.zeros(size).reshape((1, size))
    count = 0
    
    # loop on each word in the tweet
    for word in text:
        try:
            vec += word2vec[word].reshape((1, size))
            count += 1
        except KeyError:
            continue
    if count != 0:
        vec = vec / count
    return vec


In [ ]:
from sklearn.preprocessing import scale

# build our train vectors for each tweet
train_vecs = np.concatenate([TweetVector(tweet, dim) for tweet in X])
train_vecs = scale(train_vecs)



###### Let's now preprocess our test tweets and build our test vectors to predict (for kaggle)

In [ ]:
tokenizer = TweetTokenizer()

test_tweets, count_test = preprocess("twitter-datasets/test_data.txt")

# create dataframe with test  tweets
test_df = pd.DataFrame(data = {'Tweets': test_tweets})

# split each tweets into arrays of words
test_df['Tweets'] = test_df['Tweets'].apply(lambda x : tokenizer.tokenize(x))

x_test = test_df['Tweets'].values

In [ ]:
# Train word2vec on test tweets
x_test = load('tweets_emb_nltk_test.npy')
word2vec.train(x_test)

# Build test tweet vectors then scale
test_vecs = np.concatenate([TweetVector(tweet, dim) for tweet in x_test])
test_vecs = scale(test_vecs)